<a href="https://colab.research.google.com/github/thesalmonification/DSCI400_Revamp/blob/master/1DDataAndLabelMatrixMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

Google Collab Import

In [0]:
#Written in Google Collab: add drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load both Pos/Neg Session Names

In [0]:
#Load the pos/emo sessions
PosEmoSessions = list(pd.read_csv('/content/drive/Shared drives/DSCI400_Revamp/PosEmoSessions.csv',header=None)[0])
NegEmoSessions = list(pd.read_csv('/content/drive/Shared drives/DSCI400_Revamp/NegEmoSessions.csv',header=None)[0])

#print(PosEmoSessions)
#print(NegEmoSessions)

#Concatenate all pos/neg emo sessions
all_sessions = PosEmoSessions + NegEmoSessions

Create the data_tensor and label_array

In [0]:
#Create the data and label matrices
#data matrix is a 3-d tensor; label matrix will be 1-hot codes

#I will be concatenating to lists/arrays. Initialize these structures using the
#first session from all_sessions

#Further remap as binary if pos:0, if neg:1
to_binary_dict = {'0':0,'4':0,'6':0,'11':0,'1':1,'2':1,'3':1,'5':1,'12':1}

label_data = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Label_Data.h5',key=all_sessions[0]+'_labels').T
label_list = [to_binary_dict[label_data['feltEmo'].to_numpy()[0]]] * 32

waveform_array = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Waveform_Vocoded_Equalized_Downsampled_Data.h5',key=all_sessions[0]).T.to_numpy()

#1D fix
data_tensor = np.reshape(waveform_array,(32,6080))


for session in all_sessions[1:]:

  #Grab the feltEmoValue; possible numbers are ->
  #['0','4','6','11'] #Neutral, Joy/Happy, Surprise, Amusement
  #['1','2','3','5','12'] #Anger, Disgust, Fear, Sadness, Anxiety
  label_data = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Label_Data.h5',key=session+'_labels').T
  feltEmo_val = label_data['feltEmo'].to_numpy()[0]
  label_list = label_list + [to_binary_dict[feltEmo_val]] * 32
  
  #Grab the waveform matrix; reshape to be tensor
  #concatenate to the data tensor
  waveform_array = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Waveform_Vocoded_Equalized_Downsampled_Data.h5',key=session).T.to_numpy()
  waveform_array = np.reshape(waveform_array,(32,6080))
  data_tensor = np.concatenate((data_tensor,waveform_array),axis=0)



data_tensor = np.reshape(data_tensor,(16224,6080,1))
print(data_tensor.shape)
label_array = np.array(label_list)
print(label_array.shape)

(16224, 6080, 1)
16224


Save data_tensor and label_array for future use

In [0]:
#Save the tensor and array as .npy files for easy access for the neural network...
np.save('/content/drive/Shared drives/DSCI400_Revamp/data_tensor_1D.npy', data_tensor)
np.save('/content/drive/Shared drives/DSCI400_Revamp/label_array_1D.npy', label_array)